In [1]:
class VPG(object):
    def __init__(self,GraphFile,VPFile):
        self.graphfile = GraphFile
        self.VPFile = VPFile
        self.VPR = []
        self.PR = []
    
    def readFile(self):
        with open(self.graphfile) as graphfile:
            self.PR = [[int(num) for num in line.split(' ')] for line in graphfile]

        with open(self.VPFile) as VPFile:
            self.VPR = [[int(num) for num in line.split(' ')] for line in VPFile]
            
    
    def PPRelation(self):
        G = dict()
        for i in range(len(self.PR)):
            G[i] = []
            for j in range(len(self.PR)):
                if M[i][j] > 0:
                    G[i] += [j]
        self.PR = G
        return self.PR
    
    def VPRelation(self):
        G = dict()
        for i in range(len(self.VPR)):
            G[i] = []
            for j in range(len(self.VPR[0])):
                if VPR[i][j] > 0:
                    G[i] += [j]
        self.VPR = G
        return self.VPR
    
    def VehicleTripMatch(self,vehicle,Trip):
        if sum([self.VPR[vehicle][trip] == 1 for trip in Trip]) == len(Trip):
            return True
        else:
            return False
    
    def ConstructGraph(self):
        return {'Vehicle-Passenger':self.VPR,'Passenger-Passenger':self.PR}
    
    def Vehicles(self):
        if len(self.VPR) == 0:
            return 0
        else:
            return [i for i in range(len(self.VPR))]
    
    def Passengers(self):
        if len(self.VPR[0]) == 0:
            return 0
        else:
            return [i for i in range(len(self.VPR[0]))]

In [18]:
import copy
import numpy as np
from graph_tool.all import *

class trip(object):
    def __init__(self,passengers):
        self.passengers = passengers
        

class VTG(VPG):
    def __init__(self,GraphFile,VPFile,capacity):
        vpg = VPG(GraphFile,VPFile)
        vpg.readFile()
        self.V = vpg.Vehicles()
        self.P = vpg.Passengers()
        self.VPR = vpg.VPR
        self.PPR = vpg.PR
        self.VPG = vpg
        self.max_capacity = capacity
        self.neighbors = dict()
        self.Graph = []
        self.VehicleTripMatrix = []
        self.TripTripMatrix = []
        self.VehicleTripGraph = Graph(directed = False)
        self.TripTripGraph = Graph(directed = False)
        self.num_vehicles = len(self.V)
        
    def VTG(self):
        Trip = [] 
        for i in range(self.max_capacity):
            Trip.append([])
            
        for p in self.P:     
            Trip[0].append(tuple([p]))
            
        for i in range(len(Trip[0])-1):
            v_1 = Trip[0][i][0]
                
            for j in range(i+1,len(Trip[0])):
                v_2 = Trip[0][j][0]
                        
                if self.PPR[v_1][v_2] == 1: 
                    Trip[1].append((v_1,v_2))
        
                
            
        for k in range(2,self.max_capacity):
            len_TTM = len(self.TripTripMatrix)
            for i in range(len(Trip[k-1])):
                T_1 = Trip[k-1][i]
                
                for j in range(i+1,len(Trip[k-1])):
                    T_2 = Trip[k-1][j]
                    T_set = set(list(T_1)+list(T_2))
                    
                        
                    if len(T_set) == (k+1):
                        flag = 0
                        for r in range(k+1):
                            T = tuple(T_set)
                            T_list = list(T_set)
                            T_set_copy = copy.copy(T_set)
                            T_set_copy.remove(T_list[r])
                            
                            if tuple(T_set_copy) in Trip[k-1]:
                                flag += 1
                        if (flag == len(T_list)) & (T not in Trip[k]):
                            Trip[k].append(T)
        
        Trips = []
        for i in range(len(Trip)):
            Trips += Trip[i]   
        self.Trips = Trips
        self.num_trips = len(Trips)
        VTv_list = self.VehicleTripGraph.add_vertex(self.num_vehicles+self.num_trips)
        TTv_list = self.TripTripGraph.add_vertex(self.num_trips)
        V_num_passengers = self.VehicleTripGraph.new_vertex_property('num_passengers')
        V_num_passengers.a = [0]*self.VehicleTripGraph.num_vertices()
        T_num_passengers = self.TripTripGraph.new_vertex_property('num_passengers')
        
        for v in self.V:
            num = self.num_vehicles
            for i in range(len(Trip[0])):
                v_1 = Trip[0][i][0]

                V_num_passengers[self.VehicleTripGraph.vertex(i+num)] = 1
                if (self.VPR[v][v_1] == 1):
                    e = self.VehicleTripGraph.add_edge(v,i+self.num_vehicles)
            num += len(Trip[0])
            
            
            for i in range(len(Trip[1])):
                (v_1,v_2) = Trip[1][i]
                num_passengers[self.VehicleTripGraph.vertex(i+num)] = 2
                if (self.VPR[v][v_1] == 1) and (self.VPR[v][v_2] == 1):
                    e = self.VehicleTripGraph.add_edge(v,i+num)
            num += len(Trip[1])
                
            for j in range(2,len(Trip)):
                for i in range(len(Trip[j])):
                    T_list = Trip[j][i]
                    
                    V_num_passengers[self.VehicleTripGraph.vertex(i+num)] = j + 1
                    if self.VPG.VehicleTripMatch(v,T_list):
                        e = self.VehicleTripGraph.add_edge(v,i+num)
                num += len(Trip[j])
        num = self.num_vehicles                                       
        for i in range(len(Trips)):
            T_num_passengers[self.TripTripGraph.vertex(i)] = len(Trips[i])
            for k in range(i+1,len(Trips)):
                if (set(Trips[i]) & set(Trips[k])) != set():
                    e = self.TripTripGraph.add_edge(i,k)
              

        return (Trips,V_num_passengers,T_num_passengers)
    
    def get_VTM(self):
        return self.VehicleTripMatrix
    
    def get_TTM(self):
        return self.TripTripMatrix

In [3]:
import numpy as np

capacity = 4
GraphFile = 'file.txt'
VPFile = 'VPfile.txt'

def matrix_to_txt(file,AdjacencyMatrix):
    np.savetxt(file,AdjacencyMatrix,delimiter = ' ')

class BranchAndBound(VTG):
    def __init__(self,GraphFile,VPFile,capacity):
        vtg = VTG(GraphFile,VPFile,capacity)
        
        (Trips,V_num_passengers,T_num_passengers) = vtg.VTG()
        self.VTGraph = vtg.VehicleTripGraph
        self.TTGraph = vtg.TripTripGraph
        self.Trips = Trips
        self.vnp = self.VTGraph.
        

vtg = VTG(GraphFile,VPFile,capacity)
Trip = vtg.VTG()
num_v = vtg.num_v

def merge_two_dict(x,y):
    z = copy.copy(x)
    z.update(y)
    return z



def SelectEdge(VTGraph,TTGraph):
    '''
    input:
        VTGraph: the vehicle-trip graph
        TTgraph: the trip-trip graph
    output:
        max_ind: the id of the selected vehicle
    '''
    (deg,ind) = MaxDegree(VTGraph,TTGraph)
    # neighbors receive a vertex's id
  #
    #print(TTGraph.vs['id'])
    index = VTGraph.vs.select(lambda v: v['id'] == ind)
    while len(index) == 0:
        remove_ind = TTGraph.vs.select(lambda v: v['id'] == ind)[0].index
        TTGraph.delete_vertices(remove_ind)
        (deg,ind) = MaxDegree(TTGraph)
        index = VTGraph.vs.select(lambda v: v['id'] == ind)
    index = index[0]
  #  print(index)
    neighbors = VTGraph.neighbors(index)
#    print(neighbors)
    max_ind = neighbors[0]
    max_deg = 0
    for i in range(len(neighbors)):
        if VTGraph.vs[neighbors[i]].degree() > max_deg:
            max_deg = VTGraph.vs[neighbors[i]].degree()
            max_ind = neighbors[i]
 #   print(max_ind)
    return (deg,ind,max_ind,TTGraph)

def RemoveEdge(VTGraph,TTGraph,t_id,v_ind):
    '''
    input:
        VTG: the vehicle-trip graph
        TTG: the trip-trip graph
        t_ind: the id of selected trip
    output:
        v_ind: the id of the selected vehicle
        t_ind: the index of the selected trip in TTG
        VTG: the vehicle-trip graph that has an edge deleted
    '''
#    t_ind = TTGraph.vs.select(lambda vertex: vertex['id'] == t_id)[0].index
    t_index = VTGraph.vs.select(lambda v: v['id'] == t_id)[0].index
    VTGraph.delete_edges((v_ind,t_index))
    return (v_ind,t_id,VTGraph,TTGraph)

def RemoveVertex(VTGraph,TTGraph,v_ind,t_id):
    '''
    input:
        VTG
        TTG
        v_ind
        t_id: id of the trip in TTG
    output:
        VTG:
        TTG:
    '''
    # index in TTG of t_ind's neighbors
    t_ind = TTGraph.vs.select(lambda vertex: vertex['id'] == t_id)[0].index
    trip_neighbors = TTGraph.neighbors(t_ind)
    
    trip_id = TTGraph.vs[trip_neighbors]['id']
    trip_index = VTGraph.vs.select(lambda v: v['id'] in trip_id)
    t_index = [v.index for v in trip_index]

#    print(v_ind)
    TTGraph.delete_vertices(trip_neighbors+[t_ind])
  #  print(new_VTG.vs['id'])
  #  print(trip_index+[t_ind,v_ind])
#    t_id = TTGraph.vs['id']
    temp_t_ind = VTGraph.vs.select(lambda v: v['id'] == t_id)[0].index

    VTGraph.delete_vertices(t_index+[temp_t_ind]+[v_ind])
    return (VTGraph,TTGraph)
        
def StronglyConnectedComponents(VTGraph,TTGraph):
    '''
    input:
        VTG
        TTG
    output:
        VTG_scc: list
        TTG_scc: list
    '''
    # SCCs is a VertexClustering Obeject
    VTG_scc = VTGraph.components().subgraphs()
    TTG_scc = []
    for vtg_cc in VTG_scc:
        scc_id = vtg_cc.vs['id']
        trip_cc = TTGraph.subgraph(TTGraph.vs.select(lambda v: v['id'] in scc_id))
        TTG_scc.append(trip_cc)
    return (VTG_scc,TTG_scc)

def BranchBound(VTGraph,TTGraph):
    '''
    input:
        VTG
        TTG
    output:
        
    '''
    # no vehicle left
#    print(len(VTGraph.es))
  #  print(VTGraph)
    if len(TTGraph.vs) == 0:
        return (0,{})
    # one trip left
    if len(TTGraph.vs) == 1:
        if len(VTGraph.vs) >= 1:
            trip_vs = TTGraph.vs[0]
            num_passengers = trip_vs['num_passengers']
            trip = trip_vs['id']
            trip_v = VTGraph.vs.select(lambda v: v['id'] == trip)[0].index
            neighbors = VTGraph.neighbors(trip_v)
            if len(neighbors) == 0:
                TTGraph.delete_vertices(trip_vs)
                VTGraph.delete_vertices(trip_v)
                return (0,{})
            else:
                vehicle = neighbors[0]
                return (num_passengers,{trip:vehicle})
        else:
            return (0,{})
    
    if len(VTGraph.vs) == 1:
        v = VTGraph.vs[0]
        v_id = v['id']
        neighbors = VTGraph.neighbors(v)
        if len(neighbors) != 0:
            max_ind = 0
            max_num = 0
            for i in range(len(neighbors)):
                num = VTGraph.vs[neighbors[i]]['num_passengers']
                if num > max_num:
                    max_num = num
                    max_ind = i
            id = VTGraph.vs[neighbors[max_ind]]['id']
            return (max_num,{id:v_id})
        else:
            return (0,{})
    
    (vtg_scc,ttg_scc) = StronglyConnectedComponents(VTGraph,TTGraph)
    if len(vtg_scc) > 1:
        S = dict()
        W = 0
        for i in range(len(vtg_scc)):
            
            # for each connected component
            (w,s) = BranchBound(vtg_scc[i],ttg_scc[i])
            W += w
            S = merge_two_dict(S,s)
        return (W,S)
    else:
        (deg,ind,v_ind,TTGraph) = SelectEdge(VTGraph,TTGraph)
       #  if degree = 0
        if deg == 0:
            num_vehicles = sum(VTGraph.vs['type'])
            MaxMatch = VTGraph.maximum_bipartite_matching()
            Match = MaxMatch.matching
          #  print(Match)
            Graph = MaxMatch._graph
          #  print(Graph)
            match = dict()
            num = 0
            for i in range(num_vehicles):
                vehicle = Graph.vs[i]['id']
             #   print(Match[i])
                if Match[i] != -1:
                    trip = Graph.vs[Match[i]]
                    trip_id = trip['id']
                    trip_num_passengers = trip['num_passengers']
                    match[trip_id] = vehicle
                    num += trip_num_passengers
            return (num,match)           
        new_VTGraph = copy.copy(VTGraph)
        new_TTGraph = copy.copy(TTGraph)
        (v_id,t_ind,VTG_1,TTG_1) = RemoveEdge(VTGraph,TTGraph,ind,v_ind)
        (VTG_2,TTG_2) = RemoveVertex(new_VTGraph,new_TTGraph,v_id,t_ind)
        (w_1,S_1) = BranchBound(VTG_1,TTG_1)
        (w_2,S_2) = BranchBound(VTG_2,TTG_2)
        trip = TTGraph.vs.find(lambda vertex: vertex['id'] == t_ind)
        if w_2 + trip['num_passengers'] >= w_1:
            S_2 = merge_two_dict(S_2,{t_ind:v_id})
            return (w_2+trip['num_passengers'],S_2)
        else:
            return (w_1,S_1)

In [4]:
from graph_tool.all import *

/home/zeren/anaconda3/lib/python3.6/site-packages/graph_tool/all.py:40: RuntimeWarning: Error importing draw module, proceeding nevertheless: libiconv.so.2: cannot open shared object file: No such file or directory
  warnings.warn(msg, RuntimeWarning)


In [16]:
from graph_tool.all import *
g = Graph()
vlist = g.add_vertex(10)
v1 = g.add_vertex()
v2 = g.add_vertex()
print(int(v2))
g.remove_vertex(v1)
print(int(v2))
elist = g.add_edge(1,2)
e1 = g.add_edge(1,3)
e2 = g.add_edge(3,4)
print(g.edge_index[e2])
print(g.edge_index_range)
g.remove_edge(e1)
print(g.edge_index[e2])
print(g.edge_index_range)

11
11
2
3
2
3


3